# Notas sobre *machine learning*: decisão e modelos logísticos

Decisão é uma das mais populares atividades em *machine learning*. Ela compreende na construção de uma função estimadora que associa uma amostra de um determinado valor binário:

$f(x) = c_i \in \{0, 1\}$

Classificação é uma atividade ainda mais complexa, que associa amostras à elementos de um conjunto de classes. Isto é, à um inteiro que codifica a classe:

$f(x) = c_i \in \mathbb{N}$

## Introdução

No *notebook* passado, nós vimos como estimar um número a partir de um conjunto de características. Considerando o que nós sabemos até então: seria possível reaplicar tudo a fim de classificar amostras?

In [ ]:
#@title

!pip -q install shap

import os
from functools import reduce

import numpy as np
import pandas as pd
import tensorflow as tf
import shap
from sklearn.preprocessing import LabelEncoder, StandardScaler

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from google.colab import drive

sns.set(palette=sns.color_palette("hls", 8))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

drive.mount('/content/drive')

In [ ]:
WO_MEN_FILE = '/content/drive/My Drive/Colab Notebooks/ml-notes/datasets/wo_men.trusted.csv'
FEATURES = ['height', 'shoe_size']

x = pd.read_csv(WO_MEN_FILE)

In [ ]:
ss = StandardScaler()
se = LabelEncoder().fit(x.sex)

z = ss.fit_transform(x[FEATURES])
y = se.transform(x.sex)

print(z[:5].round(1))

In [ ]:
a0 = np.random.randn(2)
b0 = 0

def fn(x, a, b):
    return x.dot(a) + b

In [ ]:
As = np.linspace(-3, 2, 40)

ps = [
    metrics.mean_absolute_error(
        y,
        fn(z, np.asarray([a0[0], a]), b0)
    )
    for a in As
]

sns.lineplot(range(len(ps)), ps);

In [ ]:
pr = fn(z, a0, b0)
p = (pr >= 0).astype(int)

def acc(y, p):
    return np.mean(y == p)

print('decision fn:', pr.round(2)[:5])
print('predictions:', p.round(2)[:5])
print('accuracy:', acc(y, p).round(2))

In [ ]:
from sklearn import metrics

print('balanced accuracy:', metrics.balanced_accuracy_score(y, p).round(2))
print('accuracy:', metrics.accuracy_score(y, p).round(2))
print('Report:')
print(metrics.classification_report(y, p))

## Coletando as informações

In [ ]:
DATASET = ('/content/drive/My Drive/Colab Notebooks/ml-notes/'
           'datasets/572515_1037534_bundle_archive.zip')

In [ ]:
import zipfile
from sklearn.model_selection import train_test_split

with zipfile.ZipFile(DATASET) as z:
    z.extractall('./ds/')

t, f = (pd.read_csv('./ds/True.csv'),
        pd.read_csv('./ds/Fake.csv'))

def preprocess(t, f):
    t['target'] = 'true'
    f['target'] = 'fake'

    x = t.append(f)

    ds = x.date.str.strip()
    x['created_at'] = pd.to_datetime(ds, format='%B %d, %Y', errors='coerce')
    x.loc[x.created_at.isnull(), 'created_at'] = pd.to_datetime(ds[x.created_at.isnull()], format='%b %d, %Y', errors='coerce')
    x.loc[x.created_at.isnull(), 'created_at'] = pd.to_datetime(ds[x.created_at.isnull()], format='%d-%b-%y', errors='coerce')

    return x.dropna()

x = preprocess(t, f)

In [ ]:
def describe(x):
    print('samples:', len(x))
    print('features:', *x.columns)
    print('timeframe:', x.created_at.min().date(), x.created_at.max().date())

describe(x)

x.head(1)

## Definindo os intervalos de treinamento e teste

A separação entre treino e teste é muitas vezes realizada de forma aleatória.
É tão comum, que podemos observar utilitários para fazer isso no próprio sklearn (o [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)). O objetivo dessa estratégia é gerar uma divisão próxima à distribuição original.

Supondo que ambos subconjuntos produzidos sejam estatísticamente representativos em todas as suas características, eles apresentam o menor enviesamento e retenção de informação. Entretanto, isso pode gerar uma preocupação séria: estamos injetando informação do futuro para predizer o passado? Pense no seguinte cenário: Obama foi o presidente até 2016. Até então, haviam poucas menções ao atual presidente Trump (eleito em 2016) nas reportagens relacionadas à política.

Se nosso estimador se firmar em eventos pontuais (ex: palavras como "Obama" ou "Trump") para tomar suas decisões, podemos garantir que estes eventos irão ocorrer novamente? Como podemos avaliar a degradação de um modelo?

A separação temporal --- onde o passado é utilizado como treino e a informação mais recente é utilizada como teste --- também é uma estratégia de separação válida. Garantimos que o futuro não seja misturado com o passado e fortificamos o teste, sendo mais aderente com o cenário real.

In [ ]:
TEST_SIZE = .3
x = x.sort_values('created_at')

_at = int(len(x)*(1-TEST_SIZE))
train, test = x[:_at], x[_at:]

print(f'{len(train)} samples will be used for training.')
print('timeframe:', train.created_at.min().date(), train.created_at.max().date())
print('classes:', *zip(*np.unique(train.target, return_counts=True)), sep='\n')
print()

print(f'{len(test)} samples will be used for testing.')
print('timeframe:', test.created_at.min().date(), test.created_at.max().date())
print('classes:', *zip(*np.unique(test.target, return_counts=True)), sep='\n')

In [ ]:
plt.figure(figsize=(9, 6))
plt.subplot(221)
sns.barplot(*np.unique(train.subject, return_counts=True));
plt.xticks(rotation=45)

plt.subplot(222)
data = train.groupby(['subject', 'target'], as_index=False).count()[['subject', 'target', 'text']]
sns.barplot(x='subject', y='text', hue='target', data=data)
plt.xticks(rotation=45)

plt.subplot(223)
sns.barplot(*np.unique(train.target, return_counts=True));
plt.xticks(rotation=45)

plt.tight_layout();

## Distinguindo notícias falsas de verdadeiras a partir do texto

In [ ]:
from sklearn import metrics

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

truth_m = Pipeline([
    ('tfidf', TfidfVectorizer(strip_accents='unicode',
                              stop_words='english',
                              ngram_range=(1, 3),
                              max_features=4096,
                              lowercase=True)),
    ('lr', LogisticRegression())
])

truth_m.fit(train.text, train.target);

### Explicando o modelo

Há muitos coeficientes para serem explicados ao mesmo tempo:

In [ ]:
truth_tfidf = truth_m.named_steps['tfidf']
truth_lr = truth_m.named_steps['lr']

plt.figure(figsize=(5, 2))
sns.distplot(truth_lr.coef_);

In [ ]:
data = pd.DataFrame(truth_tfidf.transform(train.sample(1000).text).todense(),
                    columns=truth_tfidf.get_feature_names())
explainer = shap.LinearExplainer(truth_lr, data)
shap_values = explainer.shap_values(data)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0,:], data.iloc[0,:])

In [ ]:
explain(truth_fn, features=6)

In [ ]:
p = evaluate(truth_fn, test.text, test.target, test.created_at)

In [ ]:
miss = p != test.target

print('Report over missed samples')
print(f'misses: {miss.sum()} ({miss.mean():.2%})',
      dict(zip(*np.unique(test.target[miss], return_counts=True))),
      sep='\n')

Olhar para as amostras que erramos pode nos ajudar a melhorar um modelo:

In [ ]:
test[miss & (test.target == 'fake')][:3]


### Melhorando o modelo a partir de características adjuntas

Utilização de acentuação e caixa alta aconteceu em vários desses textos.
Não remover essas características, assim como as stop-words, pode melhorar o nosso modelo?

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'tfidf__lowercase': [True, False],
    'tfidf__stop_words': [None],
    'tfidf__ngram_range': [(1, 1), (2, 2), (3, 3)],
    'tfidf__strip_accents': [None],
    'tfidf__max_features': [4096],
}

truth_gr = GridSearchCV(truth_fn, params,
                        cv=3,
                        n_jobs=-1,
                        verbose=2).fit(train.text, train.target)

In [ ]:
truth_gr.best_params_

In [ ]:
explain(truth_gr.best_estimator_, features=20)

In [ ]:
p = evaluate(truth_gr, test.text, test.target, test.created_at)

In [ ]:
miss = p != test.target

print('Report over missed samples')
print(f'misses: {miss.sum()} ({miss.mean():.2%})',
      dict(zip(*np.unique(test.target[miss], return_counts=True))),
      sep='\n')

In [ ]:
test[miss & (test.target == 'fake')][['title', 'subject']]

## Inferindo o assunto da reportagem

### Aprendendo assunto a partir do título da reportagem

In [ ]:
from sklearn import metrics

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

def build_and_train_op(x, y):
    model = Pipeline([
        ('tfidf', TfidfVectorizer(strip_accents='unicode', stop_words='english',
                                  ngram_range=ngram_range, max_features=max_features)),
        ('lr', LogisticRegression())
    ])

    return model.fit(x, y)

subject_fn = build_and_train_op(train.title, train.subject)

In [ ]:
def explain(model, features=10):
    lr = model.named_steps['lr']
    coef = lr.coef_

    if coef.shape[0] == 1: # binary fb: class 0 is the opposite of class 1
        coef = np.vstack([-coef, coef])

    most_important = np.argsort(np.abs(coef), axis=1)[:, -features:]
    words = np.asarray(model.named_steps['tfidf'].get_feature_names())

    print('Total features:', len(words))

    importances = [pd.DataFrame({c: words[m], 'importance': w[m]})
                    .sort_values('importance', ascending=False)
                    .set_index(c)
                for c, w, m in zip(lr.classes_, coef, most_important)]
    
    for i in importances:
        display(i.T.round(2))

explain(subject_fn, features=6)

In [ ]:
def evaluate(model, x, y, created_at):
    p = model.predict(x)
    pa = model.predict_proba(x)
    print(metrics.classification_report(y, p))

    plt.figure(figsize=(12, 5))
    plt.subplot(121)
    show_proba_distributions(model, y, pa)
    
    if pa.shape[1] == 2:  # cm doesnt help much in binary problems
        plt.subplot(122)
        show_confusion_matrix(y, p, model.classes_)

    plt.figure(figsize=(5, 5))
    show_period_degradation(y, p, created_at)

    return p

def show_confusion_matrix(y, p, labels):
    plt.title('Confusion Matrix')
    c = metrics.confusion_matrix(y, p)
    c = c / c.sum(axis=1, keepdims=True)
    sns.heatmap(c,
                linewidths=.5, cmap='RdPu', annot=True, fmt='.0%', cbar=False,
                xticklabels=labels, yticklabels=labels);

def show_proba_distributions(model, y, pa):
    for i, c in enumerate(model.classes_):
        selected = y == c
        sns.distplot(pa[selected, i], label=c)
    
    plt.title('Prediction Probability Distributions')
    plt.legend()
    plt.tight_layout()

def show_period_degradation(y, p, created_at):
    x = pd.DataFrame({'y': y, 'p': p, 'created_at': created_at})
    r = []

    for m, t in x.groupby(created_at.dt.to_period('M')):
        r.append((m,
                  metrics.accuracy_score(t.y, t.p),
                  metrics.balanced_accuracy_score(t.y, t.p),
                  len(p)))

    r = pd.DataFrame(r, columns=['period', 'avg_accuracy', 'avg_balanced_accuracy', 'samples'])
    d = r.assign(month=r.period.astype(str))
    d = d.melt(['period', 'samples'], ['avg_accuracy', 'avg_balanced_accuracy'])

    plt.title('Estimator Metrics Over The Following Periods')
    sns.lineplot(x='period', y='value', hue='variable', data=d)
    sns.scatterplot(x='period', y='value', hue='variable', size='samples', data=d)
    plt.xticks(rotation=-70)
    plt.tight_layout()
    plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0.)

evaluate(subject_fn, test.title, test.subject, test.created_at);

### Aprendendo o assunto a partir do texto da reportagem

In [ ]:
subject_fn = build_and_train_op(train.text, train.subject)
explain(subject_fn, features=6)

In [ ]:
evaluate(subject_fn, test.text, test.subject, test.created_at);

In [ ]:
subject_fn = build_and_train_op(train.title + ' ' + train.text, train.subject)
explain(subject_fn, features=6)
evaluate(subject_fn, test.title + ' ' + test.text, test.subject, test.created_at);

#### Procurando os melhores parâmetros

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'tfidf__ngram_range': [(1, 1), (2, 2), (3, 3), (1, 3)],
    'tfidf__max_features': [4096],
}

subject_gr = GridSearchCV(
    subject_fn,
    params,
    cv=3,
    n_jobs=-1,
    verbose=1
).fit(train.title + ' ' + train.text, train.subject)

In [ ]:
subject_gr.best_params_

In [ ]:
explain(subject_gr.best_estimator_, features=6);

In [ ]:
evaluate(subject_gr, test.title + ' ' + test.text, test.subject, test.created_at);